# Feed-forward сети

Итак, давайте потренируемся тренировать нейронные сети прямого распространения (так как делали на паре)
При этом попробуем создать свою функцию активации на одном из слоев

In [1]:
# Сделаем необходимые импорты
import torch
import numpy as np

from torch import nn
import torchvision

from torch import optim

import torchvision.transforms as transforms

import torch.nn.functional as F 

from tqdm import tqdm

C:\Users\Admin\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
C:\Users\Admin\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
# Загрузим датасет CIFAR-100, сразу же создадим dataloader для него
# Если вам не хватает вычислительных ресурсов, то можно вернуться к CIFAR-10

train_dataset = torchvision.datasets.CIFAR100(root='data/',
                                             train=True,  
                                             transform=transforms.ToTensor(), 
                                             download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

test_dataset = torchvision.datasets.CIFAR100(root='./data', 
                                            train=False,
                                            download=True, 
                                            transform=transforms.ToTensor())

test_loader = torch.utils.data.DataLoader(test_dataset, 
                                          batch_size=4,
                                          shuffle=False)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting data/cifar-100-python.tar.gz to data/
Files already downloaded and verified


In [4]:
# Создайте собственную архитектуру! Можете использовать все, что угодно, но в ограничении на использование линейные слои (пока без сверток)
# Давайте добавим ограниченный Leaky_relu, то есть output = max(0.1x, 0.5x)
# Ваша задача добавить его в архитектуру сети как функцию активации

class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 6 * hidden_dim)
        self.fc2 = nn.Linear(6 * hidden_dim, 3 * hidden_dim)
        self.fc3 = nn.Linear(3 * hidden_dim, 2 * hidden_dim)
        self.fc4 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.fc5 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        tr_1 = 0.1
        tr_2 = 0.5
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x*tr_1, tr_2)
        x = self.fc2(x)
        x = F.leaky_relu(x*tr_1, tr_2)
        x = self.fc3(x)
        x = F.leaky_relu(x*tr_1, tr_2)
        x = self.fc4(x)
        x = F.leaky_relu(x*tr_1, tr_2)
        x = self.fc5(x)
        #x = F.softmax(x)
        return x
    
    def predict(self, x):
        x = x.view(x.shape[0], -1)
        
        x = self.fc1(x)
        x = F.leaky_relu(x*tr_1, tr_2)
        x = self.fc2(x)
        x = F.leaky_relu(x*tr_1, tr_2)
        x = self.fc3(x)
        x = F.leaky_relu(x*tr_1, tr_2)
        x = self.fc4(x)
        x = F.leaky_relu(x*tr_1, tr_2)
        x = self.fc5(x)
        x = F.softmax(x)
        return x


net = Net(3072, 100, 100)

In [5]:
# Запустить обучение (по аналогии с тем, что делали на паре)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.Adam(net.parameters())

for epoch in tqdm(range(10)):  
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data[0], data[1]

        # обнуляем градиент
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # выводим статистику о процессе обучения
        running_loss += loss.item()
        if i % 300 == 0:    # печатаем каждые 300 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Training is finished!')

  0%|          | 0/10 [00:00<?, ?it/s]

[1,     1] loss: 0.002
[1,   301] loss: 0.678
[1,   601] loss: 0.647


 10%|█         | 1/10 [00:21<03:12, 21.37s/it]

[2,     1] loss: 0.002
[2,   301] loss: 0.622
[2,   601] loss: 0.612


 20%|██        | 2/10 [00:44<02:59, 22.40s/it]

...

[9,     1] loss: 0.002
[9,   301] loss: 0.543
[9,   601] loss: 0.539


 90%|█████████ | 9/10 [03:25<00:23, 23.07s/it]

[10,     1] loss: 0.002
[10,   301] loss: 0.533
[10,   601] loss: 0.531


100%|██████████| 10/10 [03:48<00:00, 22.85s/it]

Training is finished!
